<a href="https://colab.research.google.com/github/synergit/adversarial-robustness-toolbox/blob/development_issue_1331/sign_opt_CIFAR10_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: http://home.mit.bme.hu/~hadhazi/Oktatas/NN18/dem3/html_demo/CIFAR-10Demo.html

In [15]:
# mount google drive https://medium.com/@ml_kid/how-to-save-our-model-to-google-drive-and-reuse-it-2c1028058cb2
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
!ls /content/gdrive

MyDrive


In [17]:
!pip install adversarial-robustness-toolbox

In [18]:
# art.utils only loads first 5000 training and 500 test data
from art.utils import load_cifar10
import matplotlib.pyplot as plt

def load_data(raw_image, length):
  (x_train, y_train), (x_test, y_test), min_, max_ = load_cifar10(raw=raw_image)#load_dataset(str("cifar10"))
  # x_train.shape (5000, 32, 32, 3)
  # y_train.shape (5000, 10)
  x_train, y_train = x_train[:length], y_train[:length]
  # x_test.shape (500, 32, 32, 3)
  # y_test.shape (500, 10)
  x_test, y_test = x_test[:int(length/10)], y_test[:int(length/10)]
  print(x_train[0].shape)
  
  imshow_CIFAR10(raw_image, x_train, y_train, 5)
  
  return (x_train, y_train), (x_test, y_test), min_, max_


def imshow_CIFAR10(raw_image, x_train, y_train, length):
  if raw_image:
    cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    print('Example training images and their labels: ' + str([x[0] for x in y_train[0:length]])) 
    print('Corresponding classes for the labels: ' + str([cifar_classes[x[0]] for x in y_train[0:length]]))

    f, axarr = plt.subplots(1, length)
    f.set_size_inches(16, 6)

    for i in range(5):
        img = x_train[i]
        axarr[i].imshow(img)
    plt.show()
  else:
    print('change `raw_image` to be True for showing images')

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

# Define the neural network model, return logits instead of activation in forward method
###
# define the network as https://arxiv.org/pdf/1608.04644.pdf, table 1
###
class Net_table1_CIFAR10(nn.Module):
    def __init__(self):
        super(Net_table1_CIFAR10, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1)
        self.conv_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.conv_3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1)
        self.conv_4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1)
        # https://discuss.pytorch.org/t/calculation-for-the-input-to-the-fully-connected-layer/82774/11
        # x.shape: torch.Size([2, 128, 5, 5]) so, 128*5*5
        self.fc_1 = nn.Linear(in_features= 3200, out_features=256) 
        self.fc_2 = nn.Linear(in_features=256, out_features=10)
        
    # https://github.com/Carco-git/CW_Attack_on_MNIST/blob/master/MNIST_Model.py
    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_3(x))
        x = F.relu(self.conv_4(x))
        x = F.max_pool2d(x, 2, 2)
        # print(x.shape) # torch.Size([2, 128, 5, 5])
        x = x.view(-1, 3200)
        
        x = F.relu(self.fc_1(x))
        x = F.dropout(x, p=0.5)
        x = self.fc_2(x)
        return x



In [20]:
# load the data

raw_image = False
(x_train, y_train), (x_test, y_test), min_, max_ = load_data(raw_image, 5000)
# Create the model

model_table1 = Net_table1_CIFAR10()

# model_scratch = Net_table1_CIFAR10()
# print(summary(model_scratch,(3, 32, 32)))


(32, 32, 3)
change `raw_image` to be True for showing images


In [21]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [22]:
import numpy as np
from art.estimators.classification import PyTorchClassifier
criterion = nn.CrossEntropyLoss()
optimizer_table1 = optim.SGD(model_table1.parameters(), lr=0.01, momentum=0.9)

# Create the ART classifier

classifier_table1 = PyTorchClassifier(
    model=model_table1,
    clip_values=(min_, max_),
    loss=criterion,
    optimizer=optimizer_table1,
    input_shape=(3, 32, 32),
    nb_classes=10,
    device_type = "gpu",
)

# Swap axes to PyTorch's NCHW format
x_train = np.transpose(x_train, (0, 3, 2, 1)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 2, 1)).astype(np.float32)

ML_model_Filename = "Pytorch_Model_CIFAR10_table1-0-1.pth"
path = F"/content/gdrive/MyDrive/{ML_model_Filename}"

try:
    with open(path, 'rb') as file:  
        # classifier_table1 = pickle.load(file)
        classifier_table1 = torch.load(file)
except FileNotFoundError:
    print('No existing model, training the model instead')
    classifier_table1.fit(x_train, y_train, batch_size=128, nb_epochs=50)
    # Save the model
    with open(path, 'wb'):  
        # pickle.dump(classifier_table1, file)
        torch.save(classifier_table1, path)

# print(classifier_table1) 

# Step 5: Evaluate the ART classifier on benign test examples
predictions = classifier_table1.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 45.4%
